#### Transform Data

In [1]:
# Import Libraries 
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential 
from azure.ai.ml import MLClient
import pandas as pd

In [2]:
# get connection to workspace 
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
    
# get token 
ws = MLClient.from_config(credential=credential)
print(ws)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f1cc28f9f90>,
         subscription_id=567f42ad-44d7-4850-aada-1d69d5b9aae9,
         resource_group_name=dp-100rg,
         workspace_name=projectomegadev)


#### Read data in pandas

In [3]:
# read data from selected files to pandas dataframe
data_path = '/home/azureuser/cloudfiles/code/Users/deborahashante/MLProject/data/' 

crops_production_df = pd.read_csv(data_path+'Crops production indicators - FAOSTAT_data_en_2-22-2024.csv')
food_trade_df = pd.read_csv(data_path+'Food trade indicators - FAOSTAT_data_en_2-22-2024.csv')
food_balance_df = pd.read_csv(data_path+"Food balances indicators - FAOSTAT_data_en_2-22-2024.csv")
land_use_df = pd.read_csv(data_path+"Land use - FAOSTAT_data_en_2-22-2024.csv")
exchange_rate_df = pd.read_csv(data_path+"Exchange rate - FAOSTAT_data_en_2-22-2024.csv")

/tmp/ipykernel_21270/1699354301.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  land_use_df = pd.read_csv(data_path+"Land use - FAOSTAT_data_en_2-22-2024.csv")


#### Transform & select relevant features

In [6]:
# function that returns a simplified dataframe for each dataset
def prepare_dataset(df, value_columns, rename_columns):
    """
    Prepares a dataset by selecting the 'Area', 'Year', and specified value columns.
    Renames the selected columns for clarity.
    """
    df = df[['Area', 'Year'] + value_columns].copy()
    df = df.rename(columns=rename_columns).reset_index(drop=True)
    return df

In [7]:
# filter the Item Column for only crop based products
crops = ['Cereals and Preparations','Fruit and Vegetables','Tobacco','Other food']
# get export value value of food traded
food_trade_filtered = prepare_dataset(food_trade_df[(food_trade_df['Element']=='Export Value') & (food_trade_df['Item'].isin(crops))], ['Value'], {'Value': 'Export_Value'})
food_trade_filtered = food_trade_filtered[(food_trade_filtered['Export_Value']>0.00)]

In [8]:
# select Land Areas specifically used for Agriculture
area = ['Agricultural land','Cropland','Agriculture','Arable land','Temporary crops','Permanent crops','Land area equipped for irrigation',
       'Land area actually irrigated','Agriculture area actually irrigated']
# get value of total land for Agriculture
land_use_filtered = prepare_dataset(land_use_df[(land_use_df['Item'].isin(area))], ['Value'], {'Value': 'Land_use'})
land_use_filtered = land_use_filtered[(land_use_filtered['Land_use']>0.00)]

In [9]:
# get crop based products that were exported
items = ['Cereals - Excluding Beer','Starchy Roots','Sugar Crops','Pulses','Treenuts','Spices','Vegetables','Oilcrops','Fruits - Excluding Wine']
# get exported quantity from food balance data
food_balance_filtered = prepare_dataset(food_balance_df[(food_balance_df['Element'] == 'Export Quantity') & (food_balance_df['Item'].isin(items))], ['Value'], {'Value': 'Export_Quantity'})
food_balance_filtered = food_balance_filtered[(food_balance_filtered['Export_Quantity']>0.00)]

In [10]:
# get value for total crop production dataset
crop_production_filtered = prepare_dataset(crops_production_df, ['Value'], {'Value': 'Total_Production'})
crop_production_filtered = crop_production_filtered[(crop_production_filtered['Total_Production']>0.00)]

In [11]:
# get value for Exchange Rate
exchange_rate_filtered = prepare_dataset(exchange_rate_df, ['Value'], {'Value': 'Exchange_Rate'})

##### Aggregate Dataset & Merge data

In [12]:
# Some data were recorded for different Items in same year, I will group and sum or average them.
crop_production = crop_production_filtered.groupby(["Area", "Year"])["Total_Production"].sum().round(2).reset_index()

# get average land use for agriculture per year for each country
land_use = land_use_filtered.groupby(["Area", "Year"])["Land_use"].mean().round(2).reset_index()

# get the sum value of food trade
food_trade = food_trade_filtered.groupby(["Area", "Year"])["Export_Value"].sum().round(2).reset_index()

# get value of total exported quantity of crop products
food_balance = food_balance_filtered.groupby(["Area", "Year"])["Export_Quantity"].sum().round(2).reset_index()

# get mean exchage rate per year for each country
exchange_rate = exchange_rate_filtered.groupby(["Area", "Year"])["Exchange_Rate"].mean().round(2).reset_index()

In [13]:
# merging the individual datasets
AB = pd.merge(land_use, crop_production, on=['Area', 'Year'], how='inner')
# merge food balance
AC = pd.merge(AB, food_balance, on=['Area', 'Year'], how='inner')
# merge exchange rate
AD = pd.merge(AC, exchange_rate, on=['Area', 'Year'], how='inner')
# merge food trade data
merged_data = pd.merge(AD,food_trade,on=['Area', 'Year'], how='inner')

In [14]:
# display data 
merged_data.head()

,Area,Year,Land_use,Total_Production,Export_Quantity,Exchange_Rate,Export_Value
0,Afghanistan,2010,11349.89,673334,360.0,46.45,203541.00
1,Afghanistan,2011,11337.44,683984,277.0,46.75,173841.00
2,Afghanistan,2012,11384.67,665926,198.0,50.92,159095.00
3,Afghanistan,2013,11391.44,716387,281.0,55.38,214687.00
4,Afghanistan,2014,11409.67,676465,410.0,57.25,384488.12


#### Clean Data

In [15]:
# general info of dataset
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Area              1979 non-null   object 
 1   Year              1979 non-null   int64  
 2   Land_use          1979 non-null   float64
 3   Total_Production  1979 non-null   int64  
 4   Export_Quantity   1979 non-null   float64
 5   Exchange_Rate     1979 non-null   float64
 6   Export_Value      1979 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 108.4+ KB


In [16]:
# Convert the Land_use column from int64 to float
merged_data['Land_use'] = merged_data['Land_use'].astype(float) 

# Convert the Total_Production column from int64 to float
merged_data['Total_Production'] = merged_data['Total_Production'].astype(float)

In [17]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Area              1979 non-null   object 
 1   Year              1979 non-null   int64  
 2   Land_use          1979 non-null   float64
 3   Total_Production  1979 non-null   float64
 4   Export_Quantity   1979 non-null   float64
 5   Exchange_Rate     1979 non-null   float64
 6   Export_Value      1979 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 108.4+ KB


#### Load data

In [19]:
# save cleaned dataset to feature store 
merged_data.to_csv(data_path+"export.csv",index=False)